# Fancier NLP

Continuing our study of NLP methods, this notebook is based on the [Chapter 16](https://github.com/ageron/handson-ml3/blob/main/16_nlp_with_rnns_and_attention.ipynb) notebook from the Scikit-learn book.

In [ ]:
# import stuff and check for GPU
import tensorflow as tf
import tensorflow_datasets as tfds

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

In [ ]:
# Load the IMDB dataset again
raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(16).prefetch(1)
valid_set = raw_valid_set.batch(16).prefetch(1)
test_set = raw_test_set.batch(16).prefetch(1)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteR8U9XE/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteR8U9XE/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteR8U9XE/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


## Reusing Pretrained Embeddings and Language Models
As with image processing, we can use transfer learning to try to leverage someone else' hard work, which may or may not be adventageous depending on the task.

Note: I tried to run this cell as-is from the source notebook, and encountered the following:
```
Your session crashed after using all available RAM.
```
Whoops, apparently the 14 GB T4 I was allocated isn't enough. I tried again and Colab put me in timeout for maxing out my runtime allocation.

**I don't recomment running this cell!**

❓What are some ways to reduce memory usage when training neural nets?

In [ ]:
# import os
# import tensorflow_hub as hub

# os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
# tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
# model = tf.keras.Sequential([
#     hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
#                    trainable=True, dtype=tf.string, input_shape=[]),
#     tf.keras.layers.Dense(64, activation="relu"),
#     tf.keras.layers.Dense(1, activation="sigmoid")
# ])
# model.compile(loss="binary_crossentropy", optimizer="nadam",
#               metrics=["accuracy"])
# model.fit(train_set, validation_data=valid_set, epochs=10)

Epoch 1/10
1407/1407 [==============================] - 1204s 817ms/step - loss: 0.2881 - accuracy: 0.8792 - val_loss: 0.2281 - val_accuracy: 0.9036
Epoch 2/10
1407/1407 [==============================] - 1115s 793ms/step - loss: 0.0213 - accuracy: 0.9940 - val_loss: 0.3439 - val_accuracy: 0.8936
Epoch 3/10
1407/1407 [==============================] - 237s 168ms/step - loss: 0.0016 - accuracy: 0.9998 - val_loss: 0.4254 - val_accuracy: 0.8992
Epoch 4/10
 742/1407 [==============>...............] - ETA: 23:21 - loss: 0.0015 - accuracy: 0.9997

## A bidirectional RNN for sentiment analysis
Keras makes it almost too easy to convert an RNN to bidirectional - just wrap in a [tf.keras.layers.Bidirectional](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Bidirectional) layer.


In [ ]:
vocab_size = 1000
text_vec_layer_ragged = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size, ragged=True)
text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels: reviews))
text_vec_layer_ragged(["Great movie!", "This is DiCaprio's best role."])

embed_size = 128
tf.random.set_seed(42)
ragged_model = tf.keras.Sequential([
    text_vec_layer_ragged,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(128)),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

ragged_model.compile(loss="binary_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
history = ragged_model.fit(train_set, validation_data=valid_set, epochs=5)

Epoch 1/5
1407/1407 [==============================] - 116s 77ms/step - loss: 0.4508 - accuracy: 0.7812 - val_loss: 0.3498 - val_accuracy: 0.8500
Epoch 2/5
1407/1407 [==============================] - 81s 58ms/step - loss: 0.3167 - accuracy: 0.8680 - val_loss: 0.3115 - val_accuracy: 0.8668
Epoch 3/5
1407/1407 [==============================] - 77s 55ms/step - loss: 0.2747 - accuracy: 0.8873 - val_loss: 0.3647 - val_accuracy: 0.8460
Epoch 4/5
1407/1407 [==============================] - 76s 54ms/step - loss: 0.2539 - accuracy: 0.8966 - val_loss: 0.3210 - val_accuracy: 0.8600
Epoch 5/5
1407/1407 [==============================] - 77s 55ms/step - loss: 0.2305 - accuracy: 0.9063 - val_loss: 0.3391 - val_accuracy: 0.8576


## An Encoder–Decoder Network for Neural Machine Translation

In this section we're loading a set of corresponding English/Spanish phrases and training an encoder/decoder to perform translation.

In [ ]:
from pathlib import Path
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

In [ ]:
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42)  # extra code – ensures reproducibility on CPU
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # separates the pairs into 2 lists

In [ ]:
for i in range(10):
    print(sentences_en[i], "=>", sentences_es[i])

How boring! => Qué aburrimiento!
I love sports. => Adoro el deporte.
Would you like to swap jobs? => Te gustaría que intercambiemos los trabajos?
My mother did nothing but weep. => Mi madre no hizo nada sino llorar.
Croatia is in the southeastern part of Europe. => Croacia está en el sudeste de Europa.
I have never eaten a mango before. => Nunca he comido un mango.
Tell the taxi driver to drive faster. => Decile al taxista que maneje más rápido.
Tom and I work together. => Tom y yo trabajamos juntos.
I would prefer an honorable death. => Preferiría una muerte honorable.
Tom married a much younger woman. => Tom se ha casado con una mujer mucho más joven.


In [ ]:
# Tokenize both the English and Spanish sentences, including start/end tokens for Spanish
vocab_size = 1000
max_length = 50
text_vec_layer_en = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    vocab_size, output_sequence_length=max_length)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [ ]:
text_vec_layer_en.get_vocabulary()[:10]

['', '[UNK]', 'the', 'i', 'to', 'you', 'tom', 'a', 'is', 'he']

In [ ]:
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [ ]:
# tf.constant just converts to a Tensor
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
# the _dec stuff is the actual Spanish for teacher forcing
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
# Let the model take plain strings as inputs
encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)

In [ ]:
# fairly arbitrary size for the word embeddings
# You could probably sub in pre-trained embeddings here
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

In [ ]:
# return_state means that that the output from the encoder includes the
# long-term and short-term hidden states of the LSTM
encoder = tf.keras.layers.LSTM(512, return_state=True)
# * is the unpacking operator, so this is just splitting apart the output (y(t))
# from the other hidden states (c(t) and h(t))
# Interestingly, encoder_outputs isn't actually used, we just want the hidden state
encoder_outputs, *encoder_state = encoder(encoder_embeddings)

In [ ]:
# Decoder is symmetric to the encoder, but we want to return the entire sequence
# this time (from t0 to tn)
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
# Pass the hidden state along to the decoder
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

In [ ]:
# As usual, a fully connected head that uses a softmax to find the most likely
# word in the vocabulary
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

**Warning**: the following cell will take a while to run (possibly a couple hours if you are not using a GPU).

In [ ]:
# Finally, mash it all together and train
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 114s 30ms/step - loss: 2.9148 - accuracy: 0.4275 - val_loss: 2.1841 - val_accuracy: 0.5196
Epoch 2/10
3125/3125 [==============================] - 71s 23ms/step - loss: 1.8661 - accuracy: 0.5716 - val_loss: 1.6756 - val_accuracy: 0.6056
Epoch 3/10
3125/3125 [==============================] - 97s 31ms/step - loss: 1.4449 - accuracy: 0.6475 - val_loss: 1.4307 - val_accuracy: 0.6544
Epoch 4/10
3125/3125 [==============================] - 93s 30ms/step - loss: 1.2040 - accuracy: 0.6945 - val_loss: 1.3338 - val_accuracy: 0.6721
Epoch 5/10
3125/3125 [==============================] - 68s 22ms/step - loss: 1.0373 - accuracy: 0.7282 - val_loss: 1.2947 - val_accuracy: 0.6821
Epoch 6/10
3125/3125 [==============================] - 76s 24ms/step - loss: 0.9039 - accuracy: 0.7565 - val_loss: 1.2860 - val_accuracy: 0.6845
Epoch 7/10
3125/3125 [==============================] - 70s 22ms/step - loss: 0.7906 - accuracy: 0.7819 - val_loss: 1.3011 

In [ ]:
# encode/decode one word at a time until we predict endofseq
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])  # encoder input
        X_dec = np.array(["startofseq " + translation])  # decoder input
        y_proba = model.predict((X, X_dec))[0, word_idx]  # last token's probas
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

In [ ]:
translate("I like soccer")

1/1 [==============================] - 0s 21ms/step


'me gusta el fútbol'

Nice! However, the model struggles with longer sentences:

In [ ]:
translate("I like soccer and also going to the beach")

1/1 [==============================] - 0s 55ms/step


'me gusta el béisbol y al [UNK] no me gusta ir'

## Bidirectional RNNs

Just like with the sentiment analysis model, we can wrap Bidirectional around our encoder RNN.

❓ Why can't the decoder RNN be Bidirectional?

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state=True))

In [ ]:
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
# concatenate the bidirectional hidden states to initialize a standard LSTM that's twice as big
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)

**Warning**: the following cell will take a while to run (possibly a couple hours if you are not using a GPU).

In [ ]:
# extra code — completes the model and trains it
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 131s 37ms/step - loss: 2.0890 - accuracy: 0.5518 - val_loss: 1.4866 - val_accuracy: 0.6429
Epoch 2/10
3125/3125 [==============================] - 85s 27ms/step - loss: 1.2904 - accuracy: 0.6798 - val_loss: 1.2831 - val_accuracy: 0.6805
Epoch 3/10
3125/3125 [==============================] - 80s 26ms/step - loss: 1.0793 - accuracy: 0.7214 - val_loss: 1.2018 - val_accuracy: 0.6987
Epoch 4/10
3125/3125 [==============================] - 81s 26ms/step - loss: 0.9358 - accuracy: 0.7504 - val_loss: 1.1814 - val_accuracy: 0.7033
Epoch 5/10
3125/3125 [==============================] - 79s 25ms/step - loss: 0.8169 - accuracy: 0.7762 - val_loss: 1.1872 - val_accuracy: 0.7043
Epoch 6/10
3125/3125 [==============================] - 85s 27ms/step - loss: 0.7161 - accuracy: 0.7994 - val_loss: 1.2051 - val_accuracy: 0.7031
Epoch 7/10
3125/3125 [==============================] - 82s 26ms/step - loss: 0.6290 - accuracy: 0.8200 - val_loss: 1.2410 

In [ ]:
translate("I like soccer")

'me gusta el fútbol'

## Attention Mechanisms

We need to feed all the encoder's outputs to the `Attention` layer, so we must add `return_sequences=True` to the encoder.

❓ What is the attention layer doing?

In [ ]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_sequences=True, return_state=True))

In [ ]:
# extra code – this part of the model is exactly the same as earlier
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

And finally, let's add the `Attention` layer and the output layer:

In [ ]:
attention_layer = tf.keras.layers.Attention()
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(attention_outputs)

**Warning**: the following cell will take a while to run (possibly a couple hours if you are not using a GPU).

In [ ]:
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 118s 33ms/step - loss: 2.0368 - accuracy: 0.5608 - val_loss: 1.4696 - val_accuracy: 0.6484
Epoch 2/10
3125/3125 [==============================] - 86s 28ms/step - loss: 1.3230 - accuracy: 0.6771 - val_loss: 1.3098 - val_accuracy: 0.6810
Epoch 3/10
3125/3125 [==============================] - 81s 26ms/step - loss: 1.1543 - accuracy: 0.7118 - val_loss: 1.2440 - val_accuracy: 0.6976
Epoch 4/10
3125/3125 [==============================] - 87s 28ms/step - loss: 1.0358 - accuracy: 0.7359 - val_loss: 1.2202 - val_accuracy: 0.7044
Epoch 5/10
3125/3125 [==============================] - 81s 26ms/step - loss: 0.9425 - accuracy: 0.7552 - val_loss: 1.2178 - val_accuracy: 0.7075
Epoch 6/10
3125/3125 [==============================] - 81s 26ms/step - loss: 0.8644 - accuracy: 0.7717 - val_loss: 1.2391 - val_accuracy: 0.7059
Epoch 7/10
3125/3125 [==============================] - 80s 25ms/step - loss: 0.7979 - accuracy: 0.7864 - val_loss: 1.2569 

In [ ]:
translate("I like soccer and also going to the beach")

'me gusta el fútbol y también ir a la playa'